In [4]:
#! METODO DESACOPLADO RAPIDO
import cmath as cm
import numpy as np

In [5]:
cos = np.cos
sin = np.sin

In [6]:
def calc_P(index, V, O, Y):
    G = np.real(Y)
    B = np.imag(Y)

    i = index - 1
    p = 0
    for m in range(len(V)):
        p += V[m]*(G[i][m]*(cos(O[i] - O[m])) + B[i][m]*(sin(O[i] - O[m])))

    return V[i]*p

def calc_Q(index, V, O, Y):
    G = np.real(Y)
    B = np.imag(Y)

    i = index - 1
    q = 0
    for m in range(len(V)):
        q += V[m]*(G[i][m]*(sin(O[i] - O[m])) - B[i][m]*(cos(O[i] - O[m])))

    return V[i]* q

def calc_Hkk(index, V, O, Y):
    G = np.real(Y)
    B = np.imag(Y)

    i = index - 1 

    h = -B[i][i]*V[i]**2 - calc_Q(index, V, O, Y)
    return h

def calc_Nkk(index, V, O, Y):
    G = np.real(Y)
    B = np.imag(Y)

    i = index - 1 

    n = (V[i]**-1)*(calc_P(index, V, O, Y) + G[i][i]*(V[i]**2))
    return n

def calc_Mkk(index, V, O, Y):
    G = np.real(Y)
    B = np.imag(Y)

    i = index - 1 

    mm = -G[i][i]*(V[i]**2) + calc_P(index, V, O, Y)
    return mm

def calc_Lkk(index, V, O, Y):
    G = np.real(Y)
    B = np.imag(Y)

    i = index - 1 

    l = (V[i]**-1)*(calc_Q(index, V, O, Y) - B[i][i]*(V[i]**2))
    return l


In [61]:
z12 = z21 = -(.01 + .01j)
z13 = z31 = -(.02 + .02j)
z23 = z32 = -(.03 + .03j)

z11 = (.03 + .03j)
z22 = (.04 + .04j)
z33 = (.05 + .05j)

Y = np.array([[1/z11, 1/z12, 1/z13], [1/z21, 1/z22, 1/z23] , [1/z31, 1/z32, 1/z33]])
V =[1, 1, 1]
O = [0, 0, 0]

P = [-.4, -.3, 0]
Q = [.2, .3, 0]

Y

array([[ 16.66666667-16.66666667j, -50.        +50.j        ,
        -25.        +25.j        ],
       [-50.        +50.j        ,  12.5       -12.5j       ,
        -16.66666667+16.66666667j],
       [-25.        +25.j        , -16.66666667+16.66666667j,
         10.        -10.j        ]])

In [62]:

for it in range(2):
    Pcalc1 = calc_P(1, V, O, Y)
    Qcalc1 = calc_Q(1, V, O, Y)

    dp1 = P[1 -1] - Pcalc1
    dq1 = Q[1 -1] - Qcalc1
    dPQ1 = np.array([[dp1], [dq1]])

    H = calc_Hkk(1, V, O, Y); L = calc_Lkk(1, V, O, Y); M = calc_Mkk(1, V, O, Y); N = calc_Nkk(1, V, O, Y)
    J = np.array([[H, N], [M, L]])
    Jinv = np.linalg.inv(J)

    deltas1 = np.matmul(Jinv, dPQ1)
    dO1 = deltas1[0][0]
    dV1 = deltas1[1][0]

    O[1 - 1] += dO1
    V[1 - 1] += dV1
    print(V, O)


    Pcalc2 = calc_P(2, V, O, Y)
    Qcalc2 = calc_Q(2, V, O, Y)

    dp2 = P[2 -1] - Pcalc2
    dq2 = Q[2 -1] - Qcalc2
    dPQ2 = np.array([[dp2], [dq2]])

    H = calc_Hkk(2, V, O, Y); L = calc_Lkk(2, V, O, Y); M = calc_Mkk(2, V, O, Y); N = calc_Nkk(2, V, O, Y)
    J = np.array([[H, N], [M, L]])
    Jinv = np.linalg.inv(J)

    deltas2 = np.matmul(Jinv, dPQ2)
    dO2 = deltas2[0][0]
    dV2 = deltas2[1][0]

    O[2 - 1] += dO2
    V[2 - 1] += dV2
    print(V, O)



[-0.3976000000000002, 1, 1] [-0.003999999999999968, 0, 0]
[-0.3976000000000002, 0.44328766099813266, 1] [-0.003999999999999968, 0.08239511305994504, 0]
[-0.041739532932028256, 0.44328766099813266, 1] [0.016513319047487464, 0.08239511305994504, 0]
[-0.041739532932028256, -0.6990257795147403, 1] [0.016513319047487464, 0.1697888751310496, 0]
